## Pregunta 9) Separación y fusión de datasets.
---

#### Elige una de las 2 operaciones:

a) Haz una copia del dataSet, consigue crear 2 dataSet con campos y filas separados pero que compartan un campo común, y después haz el merge.

- En nuestro caso, la separación ya estaba realizado en la pregunta 2., como parte de la limpieza (convertir el dataset a "tidy").
- Así nos queda solo la opción b).

b) Si tu dataSet está desactualizado o faltan datos de algunos años y los encuentras datos por otras fuentes, crea un nuevo conjunto de datos con algunas filas y/o alguna columna nueva. Por último, haz el merge.

- No hemos podido actualizar, ya que el origen de este ha sido creado en 2016 y no ha sido continuado.
- Hemos limpiado más el dataframe, después del merge, en la siguiente manera.




1. Importamos pandas, pathlib y leemos los dos ficheros csv separados (de la pregunta 2.).
---

In [1]:
import pandas as pd
from pathlib import Path # Path is for later..

df_cases: pd.DataFrame = pd.read_csv('../data/cleaned_csv/2-question_results/cases.csv', sep=',')
df_food : pd.DataFrame = pd.read_csv('../data/cleaned_csv/2-question_results/food.csv', sep=',')

2. Realizamos el merge.
3. Juntamos todos los casos que tienen los mismos valores:
    - "year", 
    - "month", 
    - "state", 
    - "location", 
    - "species", 
    - "status", 
    - "food"

Y calculamos la suma de "illnesses", "hospitalizations", "fatalities", como parte de merge.

---

In [2]:
# Merge
df_outbreaks_merged:  pd.DataFrame = df_cases.merge(df_food, how="inner", on='case_id')

# Suma de las columnas numericas como parte de merge.
df_outbreaks_grouped: pd.DataFrame = (df_outbreaks_merged
                                     .groupby(['year', 
                                               'month', 
                                               'state', 
                                               'location', 
                                               'species',
                                               'status',
                                               'food'], as_index=False )
                                     .agg(
                                           illnesses        = ('illnesses', 'sum'), 
                                           hospitalizations = ('hospitalizations', 'sum'), 
                                           fatalities       = ('fatalities', 'sum')       )
)

df_outbreaks_grouped

,year,month,state,location,species,status,food,illnesses,hospitalizations,fatalities
0,1998,April,California,Private Home/Residence,Salmonella enterica,Confirmed,unknown,7,0.0,0.0
1,1998,April,California,Restaurant,Clostridium perfringens,Confirmed,pate,11,0.0,0.0
2,1998,April,California,Restaurant,Clostridium perfringens,Confirmed,unspecified,11,0.0,0.0
3,1998,April,California,Restaurant,Shigella boydii,Confirmed,unknown,72,7.0,0.0
4,1998,April,Colorado,Catering Service,Virus,Suspected,salad,11,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
14398,2015,September,Washington,Restaurant,Salmonella enterica,Confirmed,unknown,2,0.0,0.0
14399,2015,September,Washington DC,Restaurant,Salmonella enterica,Confirmed,unknown,160,9.0,0.0
14400,2015,September,Wisconsin,Religious Facility,Salmonella enterica,Confirmed,unknown,10,0.0,0.0
14401,2015,September,Wisconsin,School/College/University,Campylobacter jejuni,Confirmed,chicken,46,3.0,0.0


4. Ordenamos el dataset juntado, por año y mes.
---

In [3]:
df_outbreaks_sorted: pd.DataFrame = df_outbreaks_grouped \
                                            .sort_values(by=['year', 'month'])
df_outbreaks_sorted

,year,month,state,location,species,status,food,illnesses,hospitalizations,fatalities
0,1998,April,California,Private Home/Residence,Salmonella enterica,Confirmed,unknown,7,0.0,0.0
1,1998,April,California,Restaurant,Clostridium perfringens,Confirmed,pate,11,0.0,0.0
2,1998,April,California,Restaurant,Clostridium perfringens,Confirmed,unspecified,11,0.0,0.0
3,1998,April,California,Restaurant,Shigella boydii,Confirmed,unknown,72,7.0,0.0
4,1998,April,Colorado,Catering Service,Virus,Suspected,salad,11,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
14398,2015,September,Washington,Restaurant,Salmonella enterica,Confirmed,unknown,2,0.0,0.0
14399,2015,September,Washington DC,Restaurant,Salmonella enterica,Confirmed,unknown,160,9.0,0.0
14400,2015,September,Wisconsin,Religious Facility,Salmonella enterica,Confirmed,unknown,10,0.0,0.0
14401,2015,September,Wisconsin,School/College/University,Campylobacter jejuni,Confirmed,chicken,46,3.0,0.0


5. En un objeto separado agrupamos el dataset por año, estados y sumamos el total de "illnesses" de ellos.
---

In [4]:
total_illnesses_by_year_state: pd.DataFrame = df_outbreaks_sorted.groupby(['year','state'])['illnesses'].sum()
total_illnesses_by_year_state

year  state        
1998  Alaska              3
      Arizona            58
      Arkansas            8
      California       1802
      Colorado          769
                       ... 
2015  Washington        410
      Washington DC     172
      West Virginia      31
      Wisconsin         688
      Wyoming             6
Name: illnesses, Length: 795, dtype: int64

6. El total valor de "illnesses" obtenido de la agrupación, lo adjuntamos al dataset "df_outbreaks_sorted".
   Por cada fila donde el "year" y "state" es match, añadimos el valor correspondiente al column "total_illnesses_by_year_state".
---

In [5]:
get_totals = lambda row: (total_illnesses_by_year_state.loc[ (row['year'], row['state']) ] )

df_outbreaks_sorted['total_illnesses_by_year_state']    = df_outbreaks_sorted.apply(func=get_totals, axis='columns')

7. Calculamos el porcentaje de "total_illnesses_by_year_state" y "illnesses" por cada fila.
Después renombramos el variable al "df_outbreaks_extended", para se queda más intuitivo y ya estamos.
---

In [6]:
df_outbreaks_sorted['illness_percentage_by_year_state'] = (
                        (df_outbreaks_sorted['illnesses'] / df_outbreaks_sorted['total_illnesses_by_year_state']) * 100
)

df_outbreaks_extended: pd.DataFrame = df_outbreaks_sorted
df_outbreaks_extended

,year,month,state,location,species,status,food,illnesses,hospitalizations,fatalities,total_illnesses_by_year_state,illness_percentage_by_year_state
0,1998,April,California,Private Home/Residence,Salmonella enterica,Confirmed,unknown,7,0.0,0.0,1802,0.388457
1,1998,April,California,Restaurant,Clostridium perfringens,Confirmed,pate,11,0.0,0.0,1802,0.610433
2,1998,April,California,Restaurant,Clostridium perfringens,Confirmed,unspecified,11,0.0,0.0,1802,0.610433
3,1998,April,California,Restaurant,Shigella boydii,Confirmed,unknown,72,7.0,0.0,1802,3.995560
4,1998,April,Colorado,Catering Service,Virus,Suspected,salad,11,0.0,0.0,769,1.430429
...,...,...,...,...,...,...,...,...,...,...,...,...
14398,2015,September,Washington,Restaurant,Salmonella enterica,Confirmed,unknown,2,0.0,0.0,410,0.487805
14399,2015,September,Washington DC,Restaurant,Salmonella enterica,Confirmed,unknown,160,9.0,0.0,172,93.023256
14400,2015,September,Wisconsin,Religious Facility,Salmonella enterica,Confirmed,unknown,10,0.0,0.0,688,1.453488
14401,2015,September,Wisconsin,School/College/University,Campylobacter jejuni,Confirmed,chicken,46,3.0,0.0,688,6.686047


8. Guardamos el dataset como "outbreaks_tidied_mergeded.csv" en el directorio "/data/cleaned_csv/9-question_results/".
---

In [7]:
def write_dataframe_to_csv(path: str, df: pd.DataFrame) -> None:
    "Writes out the given DF to a csv file."

    filepath: Path = Path(path)
    filepath.parent.mkdir(parents=True, exist_ok=True)

    df.to_csv(filepath, index=False)

write_dataframe_to_csv("../data/cleaned_csv/9-question_results/cases.csv", df_outbreaks_extended)